# Predspracovanie dát

In [56]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import json
from joblib import dump
import numpy as np

Vybrané metriky z EDA:

In [57]:
# Vybrané metriky sú vo final_selected_features.json, načítame ich:

SELECTED_FEATURES_JSON = 'final_selected_features.json'
with open(SELECTED_FEATURES_JSON, 'r') as f:
    SELECTED_FEATURES = json.load(f)

<div class="alert alert-block alert-warning">  
<b>Upozornenie:</b> Ak by sa v priebehu bežania digitálneho dvojčaťa menili dôležité metriky, týmto spôsobom ich neodchytíme.  
</div>

Mapovanie katégorických premenných na číselné hodnoty:

In [58]:
# Mapy pre log_type a application a UC
APP_MAP = {'0': 0, 'amf': 1, 'gmm': 2, 'udm': 3, 'smf': 4, 'upf': 5}
LOG_MAP = {'0': 0, 'registration': 1, 'number_of_sessions_or_ues': 2,
           'nothing': 3, 'remove': 4, 'error': 5 }
UC_MAP = { 'uc1': 1, 'uc2': 2, 'uc3': 3, 'uc4': 4, 'uc5': 5, 'uc6': 6}

In [59]:
def preprocess_df(df: pd.DataFrame, scaler=None, fit_scaler=False):
    df = df.copy()

    # Chýbajúce hodnoty
    df.fillna(df.mode().iloc[0], inplace=True)

    # Timestamp
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Mapovanie pre application, log_type a current_uc
    if 'application' in df.columns:
        df['application'] = df['application'].map(APP_MAP)

    if 'log_type' in df.columns:
        df['log_type'] = df['log_type'].map(LOG_MAP)

    if 'current_uc' in df.columns:
        df['current_uc'] = df['current_uc'].map(UC_MAP)

    # Uložíme UC map do JSON súboru, aby sme ho mohli použiť neskôr na dekódovanie
    with open('uc_map.json', 'w') as f:
        json.dump(UC_MAP, f)

    # Odstránenie riadkov s chýbajúcimi hodnotami v selected features
    df.dropna(subset=SELECTED_FEATURES, inplace=True)

    # Výber selected features
    X = df[SELECTED_FEATURES]

    # Normalizácia
    if scaler is None:
        scaler = StandardScaler()

    if fit_scaler:
        X_scaled = scaler.fit_transform(X)
    else:
        X_scaled = scaler.transform(X)

    return X_scaled, df.get('current_uc'), scaler

<div class="alert alert-block alert-info">  
<b>Predspracovanie dát:</b> Dáta predspracujeme tak, ako sme to robili v súbore EDA. 
</div>

In [60]:
# Príklad použitia:
df = pd.read_csv("../synthetic_data.csv")
X_scaled, y_scaled, fitted_scaler = preprocess_df(df, fit_scaler=True)

# predpokladáme že máš X_scaled a y pripravené
np.save("X_scaled.npy", X_scaled)
np.save("y_labels.npy", y_scaled)

# Save scaler pre použitie pri real-time inferencii
dump(fitted_scaler, 'scaler.joblib')

['scaler.joblib']

<div class="alert alert-block alert-success">  
<b>'scaler.joblib':</b> Uložíme aj scaler, aby sme mohli normalizovať dáta pred ich nahratím do digitálneho dvojčaťa.
</div>